# Finetune Gemma-7B on Vertex AI


[Gemma-7b](https://huggingface.co/google/gemma-7b) is state-of-the-art open model from Google, built from the same research and technology used to create the Gemini models. It is a text-to-text, decoder-only large language model, available in English, with open weights, and is really well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Learn more about it [Welcome Gemma - Google’s new open LLM](https://huggingface.co/blog/gemma). 

In this tutorial you will learn how to finetune [google/gemma-7b](https://huggingface.co/google/gemma-7b) on Vertex AI. 


What you'll learn in this tutorial:

1. [Setup development environment](#1-setup-development-environment)
2. [Load Dataset](#2-load-dataset)
3. [Fine-tune Gemma-7b using `trl` and `SFTTrainer`](#3-fine-tune-gemma-7b-using-trl-and-sfttrainer)
4. [Evalaute and test fine-tuned model](#4-evaluate-and-test-fine-tuned-model)

## 1. Setup development environment


In this example, we will use the Vertex AI Workbench instance with A100 and the [Hugging Face Deep Learning Containers](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face). The Hugging Face PyTorch DLC comes with all important libraries, like Transformers, Datasets, PEFT, TRL and other packages pre-installed this makes it super easy to get started, since there is no need for environment management. You can now find all Hugging Face containers on [Google Cloud](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face).


**ToDo**: Add info on how to spin-up a workbench instance or small intro about Vertex AI Workbench Instance.

**ToDo**: Update the link for the image once, GPU containers are released. 


Once the instance is up and running, we can access a Jupyter environment, which we can use for preparing our dataset and launching the training.

## 2. Load the dataset 

We use the [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset, which is a refined part of the [OpenAssistant dataset]() designed specifically to train versatile chatbots. The dataset contains various questions that require generative outputs.

The data is like a question along with its answer. Further, its multi-lingual, i.e., we have questions in English and in Spanish. The dataset contains about 9.85K training instances along with 518 test instances. An example from the dataset:

```
###Human: Can you write a joke with the following setup? A penguin and a walrus walk into a bar### Assistant: A penguin and a walrus walk into a bar. The bartender looks up and says, "What is this, some kind of Arctic joke?" The penguin and walrus just look at each other, confused. Then the walrus shrugs and says, "I don't know about Arctic jokes, but we sure know how to break the ice!" The penguin rolls his eyes, but can't help but chuckle.
```


To load the `timdettmers/openassistant-guanaco` dataset, we use the load_dataset() method from the 🤗 Datasets library.

In [1]:
# Import the necessary library for loading datasets
from datasets import load_dataset

# Specify the name of the dataset
dataset_name = "timdettmers/openassistant-guanaco"

# Load the dataset from the specified name and select the "train" split
dataset = load_dataset(dataset_name, split="train")

# Select only 2500 examples for faster training
dataset = dataset.shuffle(seed=42).select(range(2500))

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


## 3. Fine-tune Gemma-7b using `trl` and `SFTTrainer`

We will use the [SFTTrainer](https://huggingface.co/docs/trl/en/sft_trainer) from  🤗 `trl` to fine-tune our model. The `SFTTrainer`  is built on top of the 🤗 Transformers `Trainer` and inherits all the core functionalities like logging, evaluation, and checkpointing, but offers additional enhancements like:

- Packing datasets for more efficient training
- PEFT (parameter-efficient fine-tuning) support including Q-LoRA
- Preparing the model and tokenizer for conversational fine-tuning (e.g. adding special tokens)

You can read about it in the [trl docs](https://huggingface.co/docs/trl/en/sft_trainer)


As, we all know LLMs are known to be large, and running or training them in consumer hardware is a huge challenge for users and accessibility. Therefore, we  are going to use [QLoRA](https://arxiv.org/abs/2106.09685), a technqiue technique to reduce the memory footprint of LLMs during finetuning, without sacrificing performance. How it works: 

- Quantize the pretrained model to 4 bits and freezing it.
- Attach small, trainable adapter layers. (LoRA)
- Finetune only the adapter layers, while using the frozen quantized model for context.

To further enhance training efficiency, we'll incorporate a recently introduced, high-performance attention mechanism `Flash Attention 2` alongside `QLoRA`. It is nicely integrated with Transformers. It is up to 3x faster than the standard attention mechanism

In [3]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
import torch
from trl import SFTTrainer

[2024-03-25 18:11:10,350] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
# Hugging Face model id
model_id = "google/gemma-7b"

In [5]:
from transformers import BitsAndBytesConfig

config = BitsAndBytesConfig(
    load_in_4bit=True,                     #  quantize the model to 4-bits when you load it
    bnb_4bit_quant_type="nf4",             # use a special 4-bit data type for weights initialized from a normal distribution
    bnb_4bit_use_double_quant=True,        # use a nested quantization scheme to quantize the already quantized weights
    bnb_4bit_compute_dtype=torch.bfloat16, # Use float16 when running on a GPU(T4, V100) where bfloat16 is not supported
)                                          # conversion from bfloat16 to float16 may lead to overflow (and opposite may lead to loss of precision)

In order to use `google/gemma-7b` you will need the Hugging Face Hub Token, so make sure to execute the following:

```bash
huggingface-cli login # The easiest way to authenticate and it saves the token on your machine. 
```

There are other ways too which can be found in the [docs](https://huggingface.co/docs/huggingface_hub/en/quick-start).

In [7]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config = config,
                                             attn_implementation = "flash_attention_2", # use flash-attention-2 for faster training
                                             device_map = "auto"
                                            ) 

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"  # to prevent warnings

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.83s/it]


For using QLoRA with SFTTrainer, we need to create our LoraConfig and pass it as an argument to the SFTTrainer.

In [8]:
from peft import LoraConfig

peft_config = LoraConfig(
    task_type="Causal_LM", 
    target_modules="all-linear", 
    inference_mode=False, 
    r=8, 
    lora_alpha=16, 
    lora_dropout=0.05
)

Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [9]:
training_args = TrainingArguments(
    output_dir = "output",               # directory to save trained model
    num_train_epochs = 1,                # number of training epochs
    learning_rate = 2e-4,                # learning rate for training
    optim="paged_adamw_8bit",            # optimizer for training
    per_device_train_batch_size = 1,     # batch size per device during training
    gradient_accumulation_steps = 4,     # Number of steps to accumulate gradients before updating the model
    logging_steps = 10,                   # log every 10 steps
    bf16 = True                          # Use float16 when running on a GPU(T4, V100) where bfloat16 is not supported
                                         # conversion from bfloat16 to float16 may lead to overflow (and opposite may lead to loss of precision)                                       
)


In [10]:
## Initialize the trl SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # field that contains the text in the dataset
    args = training_args,
    peft_config = peft_config,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 3913.18 examples/s]


In [11]:
# start training
trainer.train()

# save model
trainer.save_model()

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,2.028200
20,1.729300
30,1.629400
40,1.697200
50,1.674500
60,1.740700
70,1.559200
80,1.626400
90,1.723600
100,1.614200


KeyboardInterrupt: 

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()